In [1]:
import gensim.models.keyedvectors as word2vec
from pycocotools.coco import COCO
from scipy.spatial import distance
import numpy as np
import os
import pickle
import time

from src.features import calc_testing_image_features

Using TensorFlow backend.


In [6]:
coco2014_dir = '/archive/MyHome/Programs/git/my_research/dataset/coco_2014/'
# 타겟 데이터에 따라 경로 수정
annFile = os.path.join(coco2014_dir, 'annotations/instances_train2014.json')

coco2014_dir_train = os.path.join(coco2014_dir, 'train2014')
coco2014_dir_val = os.path.join(coco2014_dir, 'val2014')
# coco2014_dir_test = os.path.join(coco2014_dir, 'test2014')

coco_data = COCO(annFile)
ids = coco_data.getAnnIds()
annotations = coco_data.loadAnns(ids)


loading annotations into memory...
Done (t=9.34s)
creating index...
index created!


In [7]:
projections = np.load('features/projections.npz', allow_pickle=True)
pca = projections['pca'].item()
W_img = projections['W_img']
W_tag = projections['W_tag']

possible_tags = pickle.load(open('features/possible_tags.pkl', 'rb'))

model = word2vec.KeyedVectors.load_word2vec_format('features/text8_w2v_features/text.model.bin', binary=True)
tags = []
tag_features_list = []
print('Testing: get embedding of all possible tags')
for tag in possible_tags:
    tags.append(tag)
    tag_features_list.append(model[tag])
N_TAGS = len(tags)

img_info = {}
print('Testing: get all different image ids') 
for ann in annotations:
    image_id = ann['image_id']
    if image_id not in img_info:
        img_info[image_id] = coco_data.imgs[image_id]
N_TEST = len(img_info)
print('Testing: number of images = {}'.format(N_TEST))

if not os.path.isfile('test_features.npz'):
    # 타겟 데이터에 따라 디렉토리 수정
    img_ids, img_features = calc_testing_image_features(img_info, pca, W_img, coco2014_dir_val) 
else:
    test_features = np.load('test_features.npz')
    img_ids = test_features['img_ids']
    img_features = test_features['img_features']
    

print("Finish extracting test features")
W_tag = projections['W_tag']

N_RESULTS = 5
f = open('i2t_results.txt', 'w')
pos = 0
print('Testing: prediction')
start = time.time()
for image_id in img_ids:
    v_img = img_features[pos]
    scores = np.zeros(N_TAGS)
    for i in range(N_TAGS):
        tag_features = tag_features_list[i]
        v_tag = np.dot(tag_features, W_tag)
        scores[i] = distance.euclidean(v_img, v_tag)

    index = np.argsort(scores)
    info = img_info[image_id]
    f.write(info['flickr_url'] + ' ' + info['coco_url'] + '\n')
    for i in range(N_RESULTS):
        ind = index[i]
        f.write(tags[ind] + ', ')
    f.write('\n')

    pos += 1
    
end = time.time()
print('Time: {}'.format((end - start) / 60))


Testing: get embedding of all possible tags
Testing: get all different image ids
Testing: number of images = 82081


KeyError: 262284